**Ambiente de Configuração**

In [ ]:
import pandas as pd
from fastai.vision.all import *
import os
path = Path('../input/cassava-leaf-disease-classification')

**Removendo a imagens duplicadas de acordo com esta [discussão](https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198202)**

'1562043567.jpg' e '3551135685.jpg' (rótulo incorreto)

'2252529694.jpg' e '911861181.jpg' (duplicado)

In [ ]:
data = pd.read_csv(path/'train.csv')
train = data[~data['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]

**Criando Dataloaders**

In [ ]:
#path = Path('../input/cassava-leaf-disease-classification')

def get_x(r):
    return path/'train_images'/r['image_id']

def get_y(r):
    return r['label']

In [ ]:
def get_data(size=256, bs=64, data_df=train):
    block = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                      splitter=RandomSplitter(seed=42), 
                      get_x=get_x,
                      get_y=get_y, 
                      item_tfms = RandomResizedCrop(512),
                      batch_tfms = [*aug_transforms(size=size),
                                    Normalize.from_stats(*imagenet_stats)])

    
    return block.dataloaders(data_df, bs=bs)

In [ ]:
dataloader = get_data()

In [ ]:
dataloader.show_batch()

In [ ]:
#learn = cnn_learner(dataloader, resnet50, metrics=accuracy)

In [ ]:
learn = cnn_learner(dataloader, resnet50, metrics=[error_rate,accuracy], pretrained=True)

A função *lr_find ()* da biblioteca FastAI para encontrar a melhor Taxa de aprendizagem para o modelo treinar. A função lr_find () basicamente executa o modelo em várias taxas de aprendizado até encontrar aquele LR onde o gradiente da função de perda sofre a maior queda e escolhe sugeri-lo

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(10, freeze_epochs=2)

In [ ]:
learn.fine_tune(5,2.2908675418875646e-06)

In [ ]:
learn.show_results()

In [ ]:
export = learn.export(Path("/kaggle/working/model_v9.pkl"))

In [ ]:
submission_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

In [ ]:
model_path = "../input/notebook0557b79a42/model_v9.pkl" 